In [9]:
import torch
import numpy as np
from scipy.sparse import rand as sprand

In [10]:
durations = [
    [2, 5, 4, 1, 2], # recipe 0, task 0 processing times
    [5, 4, 5, 7, 5], # recipe 0, task 1 processing times
    [4, 5, 5, 4, 5], # recipe 0, task 2 processing times
    [2, 5, 4, 7, 8], # recipe 1, task 0 processing times
    [5, 6, 9, 8, 5], # recipe 1, task 1 processing times
    [4, 5, 4, 54, 5], # . . .
    [9, 8, 6, 7, 9],
    [6, 1, 2, 5, 4],
    [2, 5, 4, 2, 4],
    [4, 5, 2, 1, 5],
    [1, 5, 2, 4, 12],
    [5, 1, 2, 1, 2]
]


In [11]:
n_factors = 5
n_users = len(durations)
n_items = len(durations[0])

In [12]:
ratings = np.matrix(durations)

In [13]:
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)

    def forward(self, user, item):
        return (self.user_factors(user) * self.item_factors(item)).sum(1)


In [14]:
model = MatrixFactorization(n_users, n_items, n_factors)
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [18]:
# Sort our data
rows, cols = ratings.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]

for row, col in zip(*(rows, cols)):
    # Set gradients to zero
    optimizer.zero_grad()
    
    # Turn data into tensors
    rating = torch.FloatTensor([ratings[row, col]])
    row = torch.LongTensor([row])
    col = torch.LongTensor([col])

    # Predict and calculate loss
    prediction = model(row, col)
    loss = loss_func(prediction, rating)

    # Backpropagate
    loss.backward()

    # Update the parameters
    optimizer.step()


In [31]:
row = 5
col = 0
rating = torch.FloatTensor([ratings[row, col]])
row = torch.LongTensor([row])
col = torch.LongTensor([col])
prediction = model(row, col)
print(prediction)

tensor([-1.7518], grad_fn=<SumBackward1>)
